In [1]:
# Load model directly
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

torch.manual_seed(17)

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-small", use_fast=True, 
                                          add_prefix_space=True)
model = AutoModelForCausalLM.from_pretrained("prajjwal1/bert-small", return_dict_in_generate=True, 
                                             pad_token_id=tokenizer.eos_token_id).to(device)

/home/yoni/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [2]:
from automata_examples.man_woman_dfa import get_man_woman_automaton

automaton = get_man_woman_automaton()
automaton.alphabet.symbols


frozenset({The,
           art,
           engineering,
           man,
           maths,
           medicine,
           music,
           science,
           studied,
           woman})

In [3]:
from model_wrapper.bert_small_wrapper import BertSmallWrapper
from llm_automaton.llm_automaton_builder import LLMAutomatonBuilder

wrapper = BertSmallWrapper(model, tokenizer, device)
builder = LLMAutomatonBuilder()

In [4]:
from pythautomata.model_exporters.dot_exporters.wfa_dot_exporting_strategy import WFADotExportingStrategy

llm_automaton = builder.construct_llm_automaton(automaton, wrapper)

dot_exporter = WFADotExportingStrategy()
dot_exporter.export(llm_automaton, "llm_automaton_BERT.dot")

tensor([[ 101, 1996,  102]])
man
<class 'pythautomata.base_types.symbol.SymbolStr'>
{'input_ids': tensor([[ 101, 2158,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}
tensor([[ 101, 2158,  102]])


IndexError: index 2158 is out of bounds for dimension 0 with size 3

In [ ]:
from llm_automaton.llm_automaton_sampler import LLMAutomatonSampler

sampler = LLMAutomatonSampler(llm_automaton)

sampler.sample(10)

NameError: name 'llm_automaton' is not defined